In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
import pennylane.numpy as np
from qiskit.quantum_info import SparsePauliOp
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.algorithms import NumPyEigensolver
from time import time

## Ham Z

In [2]:
ham = PauliSumOp(SparsePauliOp(["Z"], [1.0]))

# find the eigenvalue
exact_solver = NumPyEigensolver(k=2)
exact_result = exact_solver.compute_eigenvalues(ham)
print(np.round(exact_result.eigenvalues.real, 4))

# Save ground state energy for later
gs_energy = np.round(exact_result.eigenvalues[0], 4).real
gs_energy

[-1.  1.]


-1.0

In [3]:
ham = qml.Hamiltonian([1.0], [qml.PauliZ(0)])
print(ham)

  (1.0) [Z0]


In [4]:
@qml.qnode(qml.device("default.qubit", wires=1))
def quant_fun():
    qml.RY(0.1, wires=[0])
    qml.RZ(0.1, wires=[0])
    return qml.expval(ham)

In [5]:
opt = qml.LieAlgebraOptimizer(circuit=quant_fun, stepsize=0.1)

In [6]:
for step in range(20):
    circuit, cost = opt.step_and_cost()
    print(f"Step {step} - cost {cost}")

Step 0 - cost 0.9950041652780259
Step 1 - cost 0.9902253068524046
Step 2 - cost 0.9809071668284195
Step 3 - cost 0.9628277534405488
Step 4 - cost 0.9280856228289838
Step 5 - cost 0.86255462586615
Step 6 - cost 0.7432571413480762
Step 7 - cost 0.5399335354743116
Step 8 - cost 0.23162653487718932
Step 9 - cost -0.15462075492910926
Step 10 - cost -0.5229223112965148
Step 11 - cost -0.7777700266019967
Step 12 - cost -0.9096650664249515
Step 13 - cost -0.9658224396685353
Step 14 - cost -0.9874618733411683
Step 15 - cost -0.9954549077684967
Step 16 - cost -0.9983596523739606
Step 17 - cost -0.9994089392895383
Step 18 - cost -0.999787148625411
Step 19 - cost -0.9999233644903881


In [7]:
circuit()

tensor(-0.99997241, requires_grad=True)

In [8]:
print(qml.draw(circuit)())

0: ──RY(0.10)──RZ(0.10)──ApproxTimeEvolution(0.00,0.00,-0.02,0.20,0.10)

───ApproxTimeEvolution(-0.03,0.28,0.10)──ApproxTimeEvolution(0.00,0.00,-0.04,0.39,0.10)

───ApproxTimeEvolution(0.00,-0.00,-0.05,0.54,0.10)──ApproxTimeEvolution(-0.08,0.74,0.10)

───ApproxTimeEvolution(0.00,0.00,-0.10,1.01,0.10)──ApproxTimeEvolution(-0.14,1.33,0.10)

───ApproxTimeEvolution(-0.18,1.67,0.10)──ApproxTimeEvolution(0.00,0.00,-0.22,1.93,0.10)

───ApproxTimeEvolution(-0.24,1.96,0.10)──ApproxTimeEvolution(-0.22,1.69,0.10)

───ApproxTimeEvolution(-0.00,-0.00,-0.18,1.24,0.10)──ApproxTimeEvolution(-0.12,0.82,0.10)

───ApproxTimeEvolution(-0.08,0.51,0.10)──ApproxTimeEvolution(0.00,-0.00,-0.05,0.31,0.10)

───ApproxTimeEvolution(0.00,-0.00,-0.03,0.19,0.10)──ApproxTimeEvolution(0.00,-0.00,-0.02,0.11,0.10)

───ApproxTimeEvolution(0.00,-0.00,-0.01,0.07,0.10)──ApproxTimeEvolution(-0.01,0.04,0.10)

───ApproxTimeEvolution(0.00,-0.00,-0.00,0.02,0.10)─┤  <𝓗(1.00)>


## TFIM (closed loop)

In [9]:
def get_gs_energy(lmbd, zz_coeff, n_qubits):
    ops = []
    coeffs = []

    id_op = ['I']*n_qubits

    for i in range(n_qubits):
        temp_op = id_op.copy()
        temp_op[i] = 'X'
        ops.append(''.join(temp_op))
        coeffs.append(lmbd)

    for i in range(n_qubits-1):
        temp_op = id_op.copy()
        temp_op[i] = 'Z'
        temp_op[i+1] = 'Z'
        ops.append(''.join(temp_op))
        coeffs.append(zz_coeff)

    # closed loop
    if n_qubits > 1:
        temp_op = id_op.copy()
        temp_op[n_qubits-1] = 'Z'
        temp_op[0] = 'Z'
        ops.append(''.join(temp_op))
        coeffs.append(zz_coeff)

    ham = PauliSumOp(SparsePauliOp(ops, coeffs))

    # find the eigenvalue
    exact_solver = NumPyEigensolver(k=3)
    exact_result = exact_solver.compute_eigenvalues(ham)

    # Save ground state energy for later
    gs_energy = np.round(exact_result.eigenvalues[0], 4).real
    
    return gs_energy

In [10]:
def get_tfim_hamiltonian(lmbd, zz_coeff, n_qubits):
    ops = []
    coeffs = []

    for i in range(n_qubits):
        ops.append(qml.PauliX(i))
        coeffs.append(lmbd)

    for i in range(n_qubits-1):
        ops.append(qml.PauliZ(i)@qml.PauliZ(i+1))
        coeffs.append(zz_coeff)

    # closed loop
    if n_qubits > 1:
        ops.append(qml.PauliZ(n_qubits-1)@qml.PauliZ(0))
        coeffs.append(zz_coeff)

    H_tfim = qml.Hamiltonian(coeffs, ops)
    
    return H_tfim

In [11]:
lmbd = -1.0
zz_coeff = -1.0
L = 1

In [12]:
gs_energy = get_gs_energy(lmbd, zz_coeff, L)
print("Ground state energy:", gs_energy)

ham = get_tfim_hamiltonian(lmbd, zz_coeff, L)
print(ham)

Ground state energy: -1.0
  (-1.0) [X0]


In [13]:
def quant_fun():
    for i in range(L):
        qml.RY(0.1, wires=[i])
        qml.RZ(0.1, wires=[i])
    return qml.expval(ham)

In [14]:
qnode = qml.QNode(quant_fun, device = qml.device('default.qubit', wires = L))
print(qml.draw(qnode)())
opt = qml.LieAlgebraOptimizer(circuit=qnode, stepsize=0.1)

0: ──RY(0.10)──RZ(0.10)─┤  <𝓗(-1.00)>


In [15]:
s = time()

for step in range(10):
    circuit, cost = opt.step_and_cost()
    print(f"Step {step} - cost {cost}")
    
e = time()
print("Duration:", e - s)

Step 0 - cost -0.09933466539753062
Step 1 - cost -0.4772470126341423
Step 2 - cost -0.7506372416693825
Step 3 - cost -0.8971662287572422
Step 4 - cost -0.9608318752060236
Step 5 - cost -0.9855923626395303
Step 6 - cost -0.994771930291095
Step 7 - cost -0.9981124655143226
Step 8 - cost -0.9993197804093826
Step 9 - cost -0.9997550291318096
Duration: 0.3330390453338623


In [16]:
L = 2

gs_energy = get_gs_energy(lmbd, zz_coeff, L)
print("Ground state energy:", gs_energy)

ham = get_tfim_hamiltonian(lmbd, zz_coeff, L)
print(ham)

Ground state energy: -2.8284
  (-1.0) [X0]
+ (-1.0) [X1]
+ (-1.0) [Z0 Z1]
+ (-1.0) [Z1 Z0]


In [17]:
qnode = qml.QNode(quant_fun, device = qml.device('default.qubit', wires = L))
print(qml.draw(qnode)())
opt = qml.LieAlgebraOptimizer(circuit=qnode, stepsize=0.1)

0: ──RY(0.10)──RZ(0.10)─┤ ╭<𝓗>
1: ──RY(0.10)──RZ(0.10)─┤ ╰<𝓗>


In [18]:
s = time()

for step in range(20):
    circuit, cost = opt.step_and_cost()
    print(f"Step {step} - cost {cost}")
    
e = time()
print("Duration:", e - s)

Step 0 - cost -2.178735908636304
Step 1 - cost -2.2906178314130297
Step 2 - cost -2.3923042765779674
Step 3 - cost -2.4006665195292314
Step 4 - cost -2.3733377897199626
Step 5 - cost -2.284737718942023
Step 6 - cost -2.1874237464630992
Step 7 - cost -2.0847870955482968
Step 8 - cost -2.0052742258041185
Step 9 - cost -1.9472791158762137
Step 10 - cost -1.9120103468033331
Step 11 - cost -1.8911075172089251
Step 12 - cost -1.8797902044906116
Step 13 - cost -1.8736336768308002
Step 14 - cost -1.8704280555915291
Step 15 - cost -1.8687271606033908
Step 16 - cost -1.8678648109442348
Step 17 - cost -1.8673983591173546
Step 18 - cost -1.8671816760593747
Step 19 - cost -1.8670460417587809
Duration: 60.05732274055481


In [19]:
L = 3

gs_energy = get_gs_energy(lmbd, zz_coeff, L)
print("Ground state energy:", gs_energy)

ham = get_tfim_hamiltonian(lmbd, zz_coeff, L)
print(ham)

Ground state energy: -4.0
  (-1.0) [X0]
+ (-1.0) [X1]
+ (-1.0) [X2]
+ (-1.0) [Z0 Z1]
+ (-1.0) [Z1 Z2]
+ (-1.0) [Z2 Z0]


In [20]:
qnode = qml.QNode(quant_fun, device = qml.device('default.qubit', wires = L))
print(qml.draw(qnode)())
opt = qml.LieAlgebraOptimizer(circuit=qnode, stepsize=0.1)

0: ──RY(0.10)──RZ(0.10)─┤ ╭<𝓗>
1: ──RY(0.10)──RZ(0.10)─┤ ├<𝓗>
2: ──RY(0.10)──RZ(0.10)─┤ ╰<𝓗>


In [21]:
s = time()

for step in range(20):
    circuit, cost = opt.step_and_cost()
    print(f"Step {step} - cost {cost}")
    
e = time()
print("Duration:", e - s)

Step 0 - cost -3.2681038629544554
Step 1 - cost -1.384944029642587
Step 2 - cost 0.8272670196341178
Step 3 - cost -0.8430233738678137
Step 4 - cost 1.6293015809366371
Step 5 - cost -1.3663460769554898
Step 6 - cost 1.6499223958953553
Step 7 - cost -1.7248770183018012
Step 8 - cost 1.1165999197813208
Step 9 - cost -1.5151591139298974
Step 10 - cost 1.154354309607117
Step 11 - cost -1.7249816988188953
Step 12 - cost -0.2609236292002369
Step 13 - cost 0.8487639368683261
Step 14 - cost 0.4129512995277144
Step 15 - cost -0.5174275704313136
Step 16 - cost 0.9084859816584332
Step 17 - cost -2.4328158803320528
Step 18 - cost 0.11859881565101048
Step 19 - cost 1.202530316520938
Duration: 2952.720009803772
